LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs. The process of bringing the appropriate information and inserting it into the model prompt is known as Retrieval Augmented Generation (RAG).

In [2]:
!pip install langchain --quiet
!pip install openai --quiet
!pip install tiktoken --quiet
!pip install faiss-gpu --quiet
!pip install langchain_experimental --quiet
!pip install "langchain[docarray]" --quiet

You should consider upgrading via the 'D:\Software\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'D:\Software\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'D:\Software\Python310\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
You should consider upgrading via the 'D:\Software\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'D:\Software\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'D:\Software\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install langchain-openai --quiet

You should consider upgrading via the 'D:\Software\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip install pypdf --quiet

You should consider upgrading via the 'D:\Software\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
!pip install gradio --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1 requires torch==1.13.1, but you have torch 2.2.1 which is incompatible.
sentence-transformers 2.5.1 requires transformers<5.0.0,>=4.32.0, but you have transformers 4.30.0 which is incompatible.
You should consider upgrading via the 'D:\Software\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import PyPDFLoader
import tiktoken

In [ ]:
llm_model = "gpt-3.5-turbo"

In [ ]:
filepath = 'Lean In - Sheryl Sandberg.pdf'
loader = PyPDFLoader(filepath)
docs = loader.load_and_split()
# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunked_documents = text_splitter.split_documents(docs)
data = chunked_documents

In [ ]:
# Create vector store
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = FAISS.from_documents(data, embedding=embeddings)
#vectorstore = FAISS.from_texts(data, embedding=embeddings)

In [ ]:
# Create conversation chain
llm = ChatOpenAI(temperature=0.7, model_name=llm_model)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

In [ ]:
def run_query(query):
  result = conversation_chain({"question": query})
  return result["answer"]

In [1]:
import gradio as gr

demo = gr.Interface(
    fn=run_query,
    inputs=["text"],
    outputs=["text"],
)

demo.launch(inline=True, quiet=True)

ModuleNotFoundError: No module named 'gradio'